<a href="https://colab.research.google.com/github/PutihPutih/creditscore_machine_learning/blob/main/Credit_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

required libraries

In [ ]:
# Install kaggle API library if not already installed
!pip install kagglehub

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


using kagglehub to download the dataset

In [ ]:
# Download the dataset using kagglehub
import kagglehub

# Adjust the dataset path if necessary
path = kagglehub.dataset_download("parisrohan/credit-score-classification")
print("Path to dataset:", path)


Path to dataset: /root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/versions/1


 Load the Dataset into DataFrames

In [ ]:
# Load the datasets
train_path = "/root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/versions/1/train.csv"
test_path = "/root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/versions/1/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Check the first few rows to ensure the data is loaded
print(train_df.head())
print(test_df.head())


<ipython-input-63-b2f4c1071b68>:5: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)


       ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utilization_Ratio     C

Drop Irrelevant Columns

In [ ]:
# Drop irrelevant columns
train_df = train_df.drop(['ID', 'Customer_ID', 'Name', 'Month', 'SSN'], axis=1)
test_df = test_df.drop(['ID', 'Customer_ID', 'Name', 'Month', 'SSN'], axis=1)


In [ ]:
# List of numeric columns to clean
numeric_columns = ['Age', 'Annual_Income', 'Outstanding_Debt', 'Monthly_Balance']

# Ensure columns are numeric, convert errors to NaN
for col in numeric_columns:
    train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

# Fill missing values with the mean
for col in numeric_columns:
    train_df[col].fillna(train_df[col].mean(), inplace=True)
    test_df[col].fillna(test_df[col].mean(), inplace=True)


<ipython-input-65-710d6e9ba8e0>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].mean(), inplace=True)
<ipython-input-65-710d6e9ba8e0>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [ ]:
# Print unique values from the 'Credit_History_Age' column to see if there's anything unexpected
print(train_df['Credit_History_Age'].unique())


['22 Years and 1 Months' nan '22 Years and 3 Months'
 '22 Years and 4 Months' '22 Years and 5 Months' '22 Years and 6 Months'
 '22 Years and 7 Months' '26 Years and 7 Months' '26 Years and 8 Months'
 '26 Years and 9 Months' '26 Years and 10 Months' '26 Years and 11 Months'
 '27 Years and 0 Months' '27 Years and 1 Months' '27 Years and 2 Months'
 '17 Years and 9 Months' '17 Years and 10 Months' '17 Years and 11 Months'
 '18 Years and 1 Months' '18 Years and 2 Months' '18 Years and 3 Months'
 '18 Years and 4 Months' '17 Years and 3 Months' '17 Years and 4 Months'
 '17 Years and 5 Months' '17 Years and 6 Months' '17 Years and 7 Months'
 '17 Years and 8 Months' '30 Years and 8 Months' '30 Years and 9 Months'
 '30 Years and 10 Months' '30 Years and 11 Months' '31 Years and 0 Months'
 '31 Years and 1 Months' '31 Years and 2 Months' '31 Years and 3 Months'
 '32 Years and 0 Months' '32 Years and 2 Months' '32 Years and 3 Months'
 '32 Years and 5 Months' '32 Years and 6 Months' '30 Years and 7 

In [ ]:
import re

def convert_to_months(value):
    if pd.isna(value) or not isinstance(value, str):
        return np.nan  # Handle missing values or non-string inputs

    # Extract years and months using regex
    years_match = re.search(r'(\d+)\s+Years', value)
    months_match = re.search(r'(\d+)\s+Months', value)

    # Handle cases where either part might be missing
    years = int(years_match.group(1)) if years_match else 0
    months = int(months_match.group(1)) if months_match else 0

    return years * 12 + months


In [ ]:
train_df['Credit_History_Age'] = train_df['Credit_History_Age'].apply(convert_to_months)
test_df['Credit_History_Age'] = test_df['Credit_History_Age'].apply(convert_to_months)

# Fill any remaining NaN values with the column mean
train_df['Credit_History_Age'].fillna(train_df['Credit_History_Age'].mean(), inplace=True)
test_df['Credit_History_Age'].fillna(test_df['Credit_History_Age'].mean(), inplace=True)

# Check the results
print(train_df['Credit_History_Age'].head())
print(test_df['Credit_History_Age'].head())


0    265.000000
1    221.195405
2    267.000000
3    268.000000
4    269.000000
Name: Credit_History_Age, dtype: float64
0    273.000000
1    274.000000
2    227.251175
3    276.000000
4    327.000000
Name: Credit_History_Age, dtype: float64


<ipython-input-68-8218056281be>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Credit_History_Age'].fillna(train_df['Credit_History_Age'].mean(), inplace=True)
<ipython-input-68-8218056281be>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[c

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_df[['Credit_History_Age']] = scaler.fit_transform(train_df[['Credit_History_Age']])
test_df[['Credit_History_Age']] = scaler.transform(test_df[['Credit_History_Age']])

# Verify scaling
print(train_df['Credit_History_Age'].head())
print(test_df['Credit_History_Age'].head())


0    0.460466
1    0.000000
2    0.481490
3    0.492002
4    0.502513
Name: Credit_History_Age, dtype: float64
0    0.544561
1    0.555072
2    0.063657
3    0.576096
4    1.112199
Name: Credit_History_Age, dtype: float64


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Encode categorical columns
categorical_cols = ['Occupation', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour']
encoder = OneHotEncoder(sparse_output=False)

train_encoded = encoder.fit_transform(train_df[categorical_cols])
test_encoded = encoder.transform(test_df[categorical_cols])

# Convert to DataFrame and merge with original data
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_cols))
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_cols))

train_df = pd.concat([train_df.reset_index(drop=True), train_encoded_df], axis=1).drop(columns=categorical_cols)
test_df = pd.concat([test_df.reset_index(drop=True), test_encoded_df], axis=1).drop(columns=categorical_cols)


In [ ]:
print(train_df['Credit_Score'].unique)

<bound method Series.unique of 0        2
1        2
2        2
3        2
4        2
        ..
99995    0
99996    0
99997    0
99998    1
99999    0
Name: Credit_Score, Length: 100000, dtype: int64>


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Split the data into features and target
X = train_df.drop('Credit_Score', axis=1)
y = train_df['Credit_Score']


In [ ]:
print(X.isna().sum())  # Check for any missing values
X = X.fillna(0)  # Replace NaNs with 0 or another suitable value


Age                                                       0
Annual_Income                                             0
Monthly_Inhand_Salary                                 15002
Num_Bank_Accounts                                         0
Num_Credit_Card                                           0
Interest_Rate                                             0
Num_of_Loan                                               0
Type_of_Loan                                          11408
Delay_from_due_date                                       0
Num_of_Delayed_Payment                                 7002
Changed_Credit_Limit                                      0
Num_Credit_Inquiries                                   1965
Outstanding_Debt                                          0
Credit_Utilization_Ratio                                  0
Credit_History_Age                                        0
Total_EMI_per_month                                       0
Amount_invested_monthly                 

In [ ]:
# print(X.dtypes)
X = X.apply(pd.to_numeric, errors='coerce')  # Convert to numeric if necessary


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))



              precision    recall  f1-score   support

           0       0.78      0.77      0.77      5874
           1       0.79      0.80      0.80     10599
           2       0.71      0.69      0.70      3527

    accuracy                           0.77     20000
   macro avg       0.76      0.75      0.76     20000
weighted avg       0.77      0.77      0.77     20000

